In [ ]:
!pip install tensorflow==2.16.1
!pip install keras-tuner --upgrade

In [2]:
import os
import warnings
import random
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_theme()
matplotlib.rcParams['figure.figsize'] = [4, 3]
print(tf.__version__)

2.16.1


In [3]:
def seed(SEED=0):
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

SEED = 0
seed(SEED)
warnings.filterwarnings('ignore')

In [4]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='training loss')
  plt.plot(history.history['val_loss'], label='val loss')
  plt.title('Photoperiod 4:20')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  plt.savefig('train-val.png')
  plt.show()

In [5]:
path='https://docs.google.com/spreadsheets/d/1zY9mUlwtvh0JcsmmYN5CLPtNIjWPNpIvsnf2RoMbLnY/export?format=csv'
df = pd.read_csv(path)
df.head(5)

,Time,4:20,8:16,12:12,18:06,24:00
0,0,0.000000,0.000000,0.000000,0.000000,0.000000
1,50,7.593750,6.593750,0.000000,0.000000,0.000000
2,52,12.031250,10.406250,3.296875,4.171875,1.890625
3,54,16.468750,14.218750,6.593750,8.343750,3.781250
4,56,19.953125,17.645833,7.229167,11.229167,4.484375


In [6]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    42 non-null     int64  
 1   4:20    42 non-null     float64
 2   8:16    42 non-null     float64
 3   12:12   42 non-null     float64
 4   18:06   42 non-null     float64
 5   24:00   42 non-null     float64
dtypes: float64(5), int64(1)
memory usage: 2.1 KB


In [8]:
df['04_20'] = pd.Series(['04:20' for x in range(len(df.index))])
df['08_16'] = pd.Series(['08:16' for x in range(len(df.index))])
df['12_12'] = pd.Series(['12:12' for x in range(len(df.index))])
df['18_06'] = pd.Series(['18:06' for x in range(len(df.index))])
df['24_00'] = pd.Series(['24:00' for x in range(len(df.index))])
df.tail(5)

,Time,4:20,8:16,12:12,18:06,24:00,04_20,08_16,12_12,18_06,24_00
37,124,168.1250,220.109375,175.5625,186.50000,84.87500,04:20,08:16,12:12,18:06,24:00
38,126,166.8125,256.156250,157.8750,152.00000,87.12500,04:20,08:16,12:12,18:06,24:00
39,138,158.9375,253.125000,188.6875,163.25000,105.93750,04:20,08:16,12:12,18:06,24:00
40,140,157.6250,252.619792,246.5000,168.81250,119.34375,04:20,08:16,12:12,18:06,24:00
41,144,155.0000,251.609375,324.8750,175.34375,136.15625,04:20,08:16,12:12,18:06,24:00


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    42 non-null     int64  
 1   4:20    42 non-null     float64
 2   8:16    42 non-null     float64
 3   12:12   42 non-null     float64
 4   18:06   42 non-null     float64
 5   24:00   42 non-null     float64
 6   04_20   42 non-null     object 
 7   08_16   42 non-null     object 
 8   12_12   42 non-null     object 
 9   18_06   42 non-null     object 
 10  24_00   42 non-null     object 
dtypes: float64(5), int64(1), object(5)
memory usage: 3.7+ KB


In [45]:
df1 = df[['Time', '4:20', '04_20']]
df2 = df[['Time', '8:16', '08_16']]
df3 = df[['Time', '12:12', '12_12']]
df4 = df[['Time', '18:06', '18_06']]
df5 = df[['Time', '24:00', '24_00']]
df1.head(5)

,Time,4:20,04_20
0,0,0.000000,04:20
1,50,7.593750,04:20
2,52,12.031250,04:20
3,54,16.468750,04:20
4,56,19.953125,04:20


In [46]:
df1.columns = ['time', 'H2', 'photoperiod']
df1.head(5)

,time,H2,photoperiod
0,0,0.000000,04:20
1,50,7.593750,04:20
2,52,12.031250,04:20
3,54,16.468750,04:20
4,56,19.953125,04:20


In [47]:
df2.columns = ['time', 'H2', 'photoperiod']
df2.head(5)

,time,H2,photoperiod
0,0,0.000000,08:16
1,50,6.593750,08:16
2,52,10.406250,08:16
3,54,14.218750,08:16
4,56,17.645833,08:16


In [48]:
df6 = pd.concat([df1, df2], ignore_index=True)
df6.tail(5)

,time,H2,photoperiod
79,124,220.109375,08:16
80,126,256.156250,08:16
81,138,253.125000,08:16
82,140,252.619792,08:16
83,144,251.609375,08:16


In [49]:
df3.columns = ['time', 'H2', 'photoperiod']
df3.head(5)

,time,H2,photoperiod
0,0,0.000000,12:12
1,50,0.000000,12:12
2,52,3.296875,12:12
3,54,6.593750,12:12
4,56,7.229167,12:12


In [50]:
df7 = pd.concat([df6, df3], ignore_index=True)
print(df7)

     time          H2 photoperiod
0       0    0.000000       04:20
1      50    7.593750       04:20
2      52   12.031250       04:20
3      54   16.468750       04:20
4      56   19.953125       04:20
..    ...         ...         ...
121   124  175.562500       12:12
122   126  157.875000       12:12
123   138  188.687500       12:12
124   140  246.500000       12:12
125   144  324.875000       12:12

[126 rows x 3 columns]


In [51]:
df4.columns = ['time', 'H2', 'photoperiod']
df4.head(5)

,time,H2,photoperiod
0,0,0.000000,18:06
1,50,0.000000,18:06
2,52,4.171875,18:06
3,54,8.343750,18:06
4,56,11.229167,18:06


In [52]:
df8 = pd.concat([df7, df4], ignore_index=True)
print(df8)

     time          H2 photoperiod
0       0    0.000000       04:20
1      50    7.593750       04:20
2      52   12.031250       04:20
3      54   16.468750       04:20
4      56   19.953125       04:20
..    ...         ...         ...
163   124  186.500000       18:06
164   126  152.000000       18:06
165   138  163.250000       18:06
166   140  168.812500       18:06
167   144  175.343750       18:06

[168 rows x 3 columns]


In [53]:
df5.columns = ['time', 'H2', 'photoperiod']
df5.head(5)

,time,H2,photoperiod
0,0,0.000000,24:00
1,50,0.000000,24:00
2,52,1.890625,24:00
3,54,3.781250,24:00
4,56,4.484375,24:00


In [62]:
df9 = pd.concat([df8, df5], ignore_index=True)
df9.tail(60)

,time,H2,photoperiod
150,96,85.187500,18:06
151,98,95.343750,18:06
152,100,101.734375,18:06
153,102,108.125000,18:06
154,104,108.791667,18:06
155,106,109.458333,18:06
156,108,110.125000,18:06
157,110,110.791667,18:06
158,112,111.458333,18:06
159,114,112.125000,18:06


In [64]:
df9.sort_values('H2', inplace=True)
df9

,time,H2,photoperiod
0,0,0.000000,04:20
84,0,0.000000,12:12
85,50,0.000000,12:12
126,0,0.000000,18:06
169,50,0.000000,24:00
...,...,...,...
83,144,251.609375,08:16
82,140,252.619792,08:16
81,138,253.125000,08:16
80,126,256.156250,08:16


In [65]:
df9.tail(60)

,time,H2,photoperiod
62,88,114.881250,08:16
24,96,115.578125,04:20
116,112,116.276042,12:12
25,98,117.437500,04:20
117,114,118.500000,12:12
208,140,119.343750,24:00
160,116,119.359375,18:06
63,90,121.781250,08:16
64,92,123.671875,08:16
67,98,125.031250,08:16
